# nn
> Neural Net based on `Value` nodes

In [ ]:
#| default_exp nn

In [ ]:
#| export
from micrograd_nbdev.engine import Value
import random

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

### Neuron

In [ ]:
#| export
class Module:
    "Base class"
    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0
    
    def parameters(self):
        return []

In [ ]:
#| export
class Neuron(Module):
    # Neuron is a single calculation node
    def __init__(self,
                 nin, # number of inputs (parameters) for each node
                 nonlin=True # whether to use nonlinearity
                ):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(0)
        self.nonlin = nonlin
    
    def __call__(self, x):
        activation = sum([xi*wi for (xi, wi) in zip(x, self.w)]) + self.b
        return activation.relu() if self.nonlin else activation
    
    def parameters(self):
        return self.w + [self.b]
    
    def __repr__(self):
        return f"{'ReLU' if self.nonlin  else  'Linear'} Neuron({len(self.w)})"

In [ ]:
n = Neuron(5)
n

ReLU Neuron(5)

In [ ]:
n.parameters()

[Value(data=0.22203145250805423, grad=0),
 Value(data=0.8146914543486306, grad=0),
 Value(data=0.6845945253932413, grad=0),
 Value(data=-0.4951484752904616, grad=0),
 Value(data=0.5386153533408162, grad=0),
 Value(data=0, grad=0)]

In [ ]:
n([Value(1),Value(1),Value(1),Value(1),Value(1)])

Value(data=1.7647843103002807, grad=0)

### Layer

In [ ]:
#| export
class Layer(Module):
    def __init__(self, nin, nout, **kwargs):
        self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]
    
    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        # if nout is 1 then simple return an item rather than a list on one item
        return out[0] if len(out) == 1 else out
        
    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]
    
    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"


In [ ]:
l = Layer(2, 1)

In [ ]:
l.parameters()

[Value(data=0.5815738804807384, grad=0),
 Value(data=0.05953892107866232, grad=0),
 Value(data=0, grad=0)]

In [ ]:
l([Value(2.0), Value(1.0)])

Value(data=1.2226866820401392, grad=0)

### MLP

In [ ]:
#| export
class MLP(Module):
    def __init__(self,
                 nin, # number of inputs
                 nouts # list of inputs and outputs for each subsequent Layer
                ):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1], nonlin = i != len(nouts)-1) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]
    
    def __repr__(self):
        return f"MLP of [{', '.join(str(l) for l in self.layers)}]"

In [ ]:
mlp = MLP(2, [2,3,1]); mlp

MLP of [Layer of [ReLU Neuron(2), ReLU Neuron(2)], Layer of [ReLU Neuron(2), ReLU Neuron(2), ReLU Neuron(2)], Layer of [Linear Neuron(3)]]

In [ ]:
mlp.parameters()

[Value(data=-0.3131668656144113, grad=0),
 Value(data=0.3020218414831064, grad=0),
 Value(data=0, grad=0),
 Value(data=-0.6735470120050717, grad=0),
 Value(data=0.094031831508395, grad=0),
 Value(data=0, grad=0),
 Value(data=0.2564691606271521, grad=0),
 Value(data=0.7674418743222038, grad=0),
 Value(data=0, grad=0),
 Value(data=0.7161034265746886, grad=0),
 Value(data=0.9779788997553842, grad=0),
 Value(data=0, grad=0),
 Value(data=0.7679712408338863, grad=0),
 Value(data=0.4858063431729067, grad=0),
 Value(data=0, grad=0),
 Value(data=0.24332452137798088, grad=0),
 Value(data=-0.7837191969171422, grad=0),
 Value(data=-0.8833600660267253, grad=0),
 Value(data=0, grad=0)]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()